In [19]:
from tensorflow import keras
from keras.datasets import mnist
import cv2
import numpy as np
import sys
import time
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [20]:
(AX, ay), (QX, qy) = mnist.load_data()

In [21]:
def bigger(x, y):
    xn = np.empty((x.shape))
    xs = np.empty((x.shape))
    xl = np.empty((x.shape))
    xo = np.empty((x.shape))

    norte = np.float32([[1,0,-1], [0,1,0]])      # Deslocamento para norte
    sul   = np.float32([[1,0,1], [0,1,0]])       # Deslocamento para sul
    leste = np.float32([[1,0,0], [0,1,1]])       # Deslocamento para leste
    oeste = np.float32([[1,0,0], [0,1,-1]])      # Deslocamento para oeste

    length = x.shape[0]
    for i in range(length):
        xn[i] = cv2.warpAffine(x[i], norte, (x.shape[1], x.shape[2]))
        xs[i] = cv2.warpAffine(x[i], sul, (x.shape[1], x.shape[2]))
        xl[i] = cv2.warpAffine(x[i], leste, (x.shape[1], x.shape[2]))
        xo[i] = cv2.warpAffine(x[i], oeste, (x.shape[1], x.shape[2]))

    new_x = np.append(x, xn, axis=0)
    new_x = np.append(new_x, xs, axis=0)
    new_x = np.append(new_x, xl, axis=0)
    new_x = np.append(new_x, xo, axis=0)
    
    y = np.append(y, np.array([y]))
    y = np.append(y, np.array([y]))
    y = np.append(y, np.array([y]))
    y = np.append(y, np.array([y]))
    
    return new_x, y
    

In [22]:
ax=np.empty((AX.shape[0],14,14))
for i in range(AX.shape[0]):
    ax[i]=cv2.resize(AX[i],(14,14));
qx=np.empty((QX.shape[0],14,14))
for i in range(QX.shape[0]):
    qx[i]=cv2.resize(QX[i],(14,14));

ax, ay = bigger(ax, ay)
qx, qy = bigger(qx, qy)

In [23]:
ax = ax.astype('float32')
qx = qx.astype('float32')
ax = ax.reshape(ax.shape[0],ax.shape[1]*ax.shape[2])
qx = qx.reshape(qx.shape[0],qx.shape[1]*qx.shape[2])
ax /= 255.0 #0 a 1
qx /= 255.0 #0 a 1
qp=np.empty(qy.shape,dtype="uint8")

In [24]:
t1 = time.time()
FLANN_INDEX_KDTREE = 1
flann_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 4)
flann = cv2.flann_Index(ax, flann_params)
t2 = time.time()
matches, dists = flann.knnSearch(qx, 1)
t3 = time.time()

In [25]:
for l in range(matches.shape[0]):
    i=matches[l]
    qp[l]=ay[i]

In [26]:
erros=0
for l in range(matches.shape[0]):
    if qp[l]!=qy[l]: erros+=1

In [27]:
print("Erros=%5.2f%%" % (100.0*erros/qy.shape[0]) )
print("Tempo de treinamento: %f"%(t2-t1))
print("Tempo de predicao: %f"%(t3-t2))

Erros= 1.16%
Tempo de treinamento: 5.380121
Tempo de predicao: 1.566865
